a) Python-Routine um lars seine .npy-Dateien als Diagramm darzustellen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.fft import fft, fftfreq

folders = ["hoch", "links", "rechts", "tief"]

#44.100 Abtastvorgänge pro Sekunde
def cut_signal(data, threshold=0.01, fs=44100):
    start_index = 0
    for i, sample in enumerate(data):
        if abs(sample) > threshold:
            start_index = i
            break
    end_index = start_index + fs
    cut_data = data[start_index:end_index]
    if len(cut_data) < fs:
        cut_data = np.append(cut_data, [0]*(fs - len(cut_data)))
    return cut_data

for folder in folders:
    files = [file for file in os.listdir(f"lars/{folder}") if file.endswith(".npy")]
    n = len(files)
    fig, axs = plt.subplots(n, 2, figsize=(10, n*2))
    fs = 44100

    for i, file in enumerate(files):
        data = np.load(f"andy/{folder}/" + file)
        triggered_data = cut_signal(data)

        # Plot original data
        axs[i, 0].plot(data)
        axs[i, 0].set_title(f"Original data: {folder}: {file}")
        axs[i, 0].set_xlabel('Sample')
        axs[i, 0].set_ylabel('Amplitude')

        # Plot triggered data
        axs[i, 1].plot(triggered_data)
        axs[i, 1].set_title(f"Triggered data: {folder}: {file}")
        axs[i, 1].set_xlabel('Sample')
        axs[i, 1].set_ylabel('Amplitude')

    plt.tight_layout()
    plt.show()

c) Bestimmung des Amplitudenspektrums
Stelle dieses Amplitudenspektrum graphisch dar.

In [ ]:
# Apply the function to the data
for folder in folders:
    files = [file for file in os.listdir(f"lars/{folder}") if file.endswith(".npy")]

    # Create a grid of 1 column and enough rows to hold all plots
    n = len(files)
    fig, axs = plt.subplots(n, 1, figsize=(10, n*2))
    fs = 44100

    for i, file in enumerate(files):
        data = np.load(f"lars/{folder}/" + file)
        triggered_data = cut_signal(data)

        # Compute the amplitude spectrum
        N = len(triggered_data)
        yf = fft(triggered_data)
        xf = fftfreq(N, 1 / 44100)

        axs[i].plot(xf, np.abs(yf))
        axs[i].set_title(f"{folder}: {file}")
        axs[i].set_xlabel('Frequenz (Hz)')  # Set x-axis label
        axs[i].set_ylabel('Amplitude')  # Set y-axis label

    plt.tight_layout()
    plt.show()

d) Implementierung des Windowing-Verfahrens

In [ ]:
def windowing(data, window_size=512):
    hop_size = window_size // 2

    # Erstelle eine Gaußsche Fensterfunktion mit einer Standardabweichung von 1/4 der Fenstergröße.
    window = gaussian(window_size, std=window_size / 4)
    windowed_data = []

    for i in range(0, len(data) - window_size + 1, hop_size):
        # Extrahiere einen Abschnitt (Fenster) der Größe 'window_size' aus den Daten.
        section = data[i:i + window_size]

        # Wende die Gaußsche Fensterfunktion auf den Abschnitt an, um ihn zu gewichten.
        windowed_section = section * window

        # Füge den gewichteten Abschnitt zur Liste der gefensterten Daten hinzu.
        windowed_data.append(windowed_section)

    return windowed_data



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq
from scipy.signal.windows import gaussian

# apply windowing function to the data
for folder in folders:
    files = [file for file in os.listdir(f"lars/{folder}") if file.endswith(".npy")]

    # Create a grid of 1 column and enough rows to hold all plots
    n = len(files)
    fig, axs = plt.subplots(n, 1, figsize=(10, n*2))
    fs = 44100

    for i, file in enumerate(files):
        data = np.load(f"lars/{folder}/" + file)
        triggered_data = cut_signal(data)
        windowed_data = windowing(triggered_data)

        # Compute the amplitude spectrum
        N = len(windowed_data[0])
        yf = fft(windowed_data[0])
        xf = fftfreq(N, 1 / 44100)

        axs[i].plot(xf, (yf))
        axs[i].set_title(f"{folder}: {file}")
        axs[i].set_xlabel('Frequenz (Hz)')  # Set x-axis label
        axs[i].set_ylabel('Amplitude')  # Set y-axis label

    plt.tight_layout()
    plt.show()


